In [1]:
import json
import numpy as np
import pandas as pd
import os

In [2]:


def list_keys(schema, parent_key=''):
    keys = []
    if isinstance(schema, dict):
        for key, value in schema.items():
            full_key = f"{parent_key}.{key}" if parent_key else key
            if isinstance(value, dict):
                key_type = value.get('type', 'object')
                key_format = value.get('format', '')
                key_pattern = value.get('pattern', '')
                key_description = value.get('description', '')
                keys.append((full_key, key_type, key_format, key_pattern, key_description))
                keys.extend(list_keys(value, full_key))
            elif isinstance(value, list):
                key_type = 'array'
                keys.append((full_key, key_type, '', '', ''))
                for item in value:
                    keys.extend(list_keys(item, full_key))
            else:
                key_type = 'string'
                keys.append((full_key, key_type, '', '', ''))
    return keys

# Load the JSON schema
with open('../Data/Schemas/cve_api_json_2.0.schema', 'r') as file:
    schema = json.load(file)

# List keys under "definitions"
definitions_keys = list_keys(schema.get('definitions', {}))

# Convert the list of keys into a DataFrame
df = pd.DataFrame(definitions_keys, columns=['Keys', 'Type', 'Format', 'Pattern', 'Description'])

# Convert dictionary values to strings
df = df.astype(str)

# Sort the DataFrame by 'Keys' and then by 'Type'
df = df.sort_values(by=['Keys', 'Type'])

# Create the output directory if it doesn't exist
output_dir = '../NVD Program/Files'
os.makedirs(output_dir, exist_ok=True)

# Export the sorted DataFrame to a CSV file
output_file = os.path.join(output_dir, 'NVD_Schema.csv')
df.to_csv(output_file, index=False)

In [3]:

# Replace empty strings with NaN
df.replace('', np.nan, inplace=True)

# Calculate the percentage of each column that has a value
percentages = df.notnull().mean() * 100
percentages_df = percentages.reset_index()
percentages_df.columns = ['Column', 'Percentage']

# Display the DataFrame with percentages
percentages_df

,Column,Percentage
0,Keys,100.000000
1,Type,100.000000
2,Format,2.290076
3,Pattern,0.763359
4,Description,3.816794
